In [ ]:
# use John Snow Lab T5
# joepareti54@gmail.com 10-19-2021
#

In [1]:
import json
import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

21/10/19 12:53:43 WARN Utils: Your hostname, joepareti54-GS75-Stealth-9SE resolves to a loopback address: 127.0.1.1; using 192.168.1.79 instead (on interface wlo1)
21/10/19 12:53:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/joepareti54/.ivy2/cache
The jars for the packages stored in: /home/joepareti54/.ivy2/jars
:: loading settings :: url = jar:file:/home/joepareti54/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7aca0fe6-482b-47f8-942d-0ccb0840609a;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.3.0 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in cen

In [2]:
document_assembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("documents")

t5 = T5Transformer() \
  .pretrained("t5_small", 'en') \
  .setTask("summarize:")\
  .setMaxOutputLength(200)\
  .setInputCols(["documents"]) \
  .setOutputCol("summaries")

summarizer_pp = Pipeline(stages=[
    document_assembler, t5
])

t5_small download started this may take some time.
Approximate size to download 139 MB
[ | ]t5_small download started this may take some time.
Approximate size to download 139 MB
[ \ ]Download done! Loading the resource.
[ / ]

2021-10-19 12:54:39.200099: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-19 12:54:39.285883: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599500000 Hz


[OK!]


In [3]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = summarizer_pp.fit(empty_df)
sum_lmodel = LightPipeline(pipeline_model)



In [4]:
example_txt = """ An urgent call reached Ronald O’Hanley, State Street Corp.’s chief executive, as he sat in his oﬃce in downtown Boston. It was 8 a.m. on Monday, March 16. A senior deputy told him corporate treasurers and pension managers, panicked by the growing economic damage from the Covid-19 pandemic, were pulling billions of dollars from certain money-market funds. This was forcing the funds to try to sell some of the bonds they held. But there were almost no buyers. Everybody was suddenly desperate for cash. He and the deputy, asset-management executive Cyrus Taraporevala, had spoken the night before, wrestling with how investors would respond to an emergency interest-rate cut from the Federal Reserve. Now, they had their answer. In his 34 years in ﬁnance, Mr. O’Hanley had weathered plenty of meltdowns, but never one like this. “The market is fearing the worst,” Mr. O’Hanley told him. March 16 was the day a microscopic virus brought the ﬁnancial system to the brink. Few realized how close it came to going over the edge entirely. The Dow Jones Industrial Average plunged nearly 13% that day, the secondbiggest one-day fall in history. Stock-market volatility spiked to a record high. Investors struggled to unload even safe bonds, like Treasurys. Companies and government oﬃcials were losing access to the lending markets on which they rely to make payroll and build schools.   Prime money-market funds that are owned by big institutional investors and buy a lot of short-term corporate debt—normally safe and boring—had outﬂows of $60 billion in the week ending that Wednesday, ﬁnancial-data ﬁrm Reﬁnitiv said, among the worst ever. Some $56 billion in client money ﬂed bond funds. Interest rates on short-term corporate debt surged, peaking on March 25 at 2.43 percentage points above the federal-funds rate—the highest it has been since October 2008, according to the Federal Reserve Bank of St. Louis.   The ﬁnancial system has endured numerous credit crunches and market crashes, and memories of the 1987 and 2008 crises set a high bar for market dysfunction. But longtime investors and those who make a living on Wall Street say mid-March of this year was far more severe in a short period. Moreover, the stresses to the ﬁnancial system were broader than many had seen. “The 2008 ﬁnancial crisis was a car crash in slow motion,” said Adam Lollos, head of short-term credit at Citigroup Inc. “This was like, ‘Boom!’ ” A barrage of government programs has since pulled the system back from collapse. This account of what happened on one of the worst days the ﬁnancial markets have ever seen, from many of the executives, money managers and Wall Street veterans who lived it, shows why the rescue eﬀort was so urgent. The Federal Reserve set the stage for the downturn on Sunday, March 15. Most investors were expecting the central bank to announce its latest response to the crisis the following Wednesday. Instead, it announced at 5 p.m. that evening that it was slashing interest rates and planning to buy $700 billion in bonds to help unclog the markets.   Rather than take comfort in the Fed’s actions, many companies, governments, bankers and investors viewed the decision as reason to prepare for the worst possible outcome from the coronavirus pandemic. A downdraft in bonds was now a rout. Mr. O’Hanley was in a good position to see the crisis unfold. His bank provides vital, if unheralded, administrative and bookkeeping services for most of the world’s biggest investors, and runs its own trillion-dollar money manager. Companies and pension managers have long relied on money-market funds that invest in short-term corporate and municipal-debt holdings considered safe and liquid enough to be classiﬁed as “cash equivalents.” They function almost like checking accounts—helping ﬁrms manage payroll, pay oﬃce leases and move cash around to ﬁnance their daily operations. But that Monday, investors no longer believed certain money funds were cashlike at all. As they pulled their money out, managers struggled to sell bonds to meet redemptions. In theory, there should have been some give in the system. U.S. regulators had rewritten the rules on money funds in  the wake of the 2008 ﬁnancial crisis, replacing their ﬁxed, $1 price with a ﬂoating one that moved with the value of their holdings. The changes headed oﬀ the panic that could ensue when a fund’s price “breaks the buck,” as one prominent fund had in 2008.   But the rules couldn’t stop a panicked assault like this one. Rumors circulated that some of State Street’s rivals would be forced to prop up their funds. Within days, both Goldman Sachs Group Inc. and Bank of New York Mellon Corp. stepped in to buy assets from their money funds. Both ﬁrms declined to comment. This was bad news for not only those funds and their investors, but also for the thousands of companies and communities dependent on short-term loan markets to pay their employees. “If junk bonds back up, people can rationalize that away,” Mr. O’Hanley said. “There’s very little ability to rationalize trouble in cash.” A debt-investing unit of Prudential Financial Inc., one of the largest insurance companies in the world, was also struggling with normally safe securities. When traders at PGIM Fixed Income tried that Monday to sell a batch of shortterm bonds issued by highly rated companies, they found few takers. And banks were reluctant to step in as intermediaries.  “The broker-dealer community was frozen,” said Michael Collins, a senior ﬁxedincome manager at PGIM. “It was as bad as at any point during the great ﬁnancial crisis.” Across the country in Southern California, the head of the debt-trading desk at investment ﬁrm Capital Group Cos., Vikram Rao, tried to make sense of the dysfunction. Mr. Rao, who was working remotely that Monday, walked down the 20 steps to his home oﬃce at 4:30 a.m. to discover the debt markets were already in disarray. He started calling the senior Wall Street executives he knew at many of the big banks. Executives told him that Sunday’s emergency Fed rate cut had swung a swath of interest-rate swap contracts in banks’ favor. Companies had locked in superlow interest rates on future debt sales over the past year. But when rates fell even further, the companies suddenly owed additional collateral. On that Monday, banks had to account for all that new collateral as assets on their books. So when Mr. Rao called senior executives for an explanation on why they wouldn’t trade, they had the same refrain: There was no room to buy bonds and other assets and still remain in compliance with tougher guidelines imposed by regulators after the previous ﬁnancial crisis. In other words, capital rules intended to make the ﬁnancial system safer were, at least in this instance, draining liquidity from the markets.   One senior bank executive leveled with him: “We can’t bid on anything that adds to the balance sheet right now.” At the same time, the surge in stockmarket volatility, along with falling prices on mortgage bonds, had forced margin calls on many investment funds. The additional collateral they owed banks was also booked as assets, adding billions more. The slump in mortgage bonds was so vast it crushed a group of investors that had borrowed from banks to juice their returns: real-estate investment funds. The Fed’s bond-buying program, unveiled that Sunday, had earmarked some $200 billion for mortgage-bond purchases. But by Monday bond  while well-intentioned, weren’t nearly  managers discovered the Fed purchases, enough.   “On that ﬁrst day, the Fed got completely run over by the market,” said Dan Ivascyn, who manages one of the world’s biggest bond funds and serves as investment chief at Paciﬁc Investment Management Co. “That’s where REITs  and other leveraged-mortgage products started getting into serious trouble.” That Tuesday, UBS Group AG closed two exchange-traded notes tied to mortgage real-estate investment trusts. By Friday, a mortgage trust run by hedge-fund ﬁrm Angelo Gordon & Co. had warned its lenders it wouldn’t be able to meet its obligations on future margin calls.   For decades investors have eagerly scooped up state and local government bonds month after month, week after week, every week. But that came to a standstill in mid-March. Terriﬁed investors ditched municipal debt at ﬁre-sale prices, underwriters canceled billions of dollars worth of deals and new borrowing stopped. There was less bond issuance in the week of March 16 than at any point during the 2008 ﬁnancial crisis, the 2001 terrorist attacks or the week of 1987’s Black Monday, according to Reﬁnitiv data, adjusted for inﬂation. For those few days in March, investors lost faith in America’s public infrastructure. As schools and universities shut down and airports and public transit systems emptied out, the market began to question what had been previously considered gold-plated bets on the core institutions that make up community life in the country. The deep trouble in the market was clear early on the morning of March 16. Cities and states often rely on short-term debt issued through bond dealers, who then resell the securities to investors. Billions of dollars of that paper was up for resale the following day. Rates, which had been around 1.28%, looked like they could reach 6%.   At the same time, long-term municipal-bond deals were being pulled. Over the course of the week Citigroup Inc., the second-biggest underwriter in the municipal market, wouldn’t launch a single new bond. Staﬀ at Citigroup’s municipal markets division worked in various locations that day, some from home, some from the bank’s Manhattan headquarters and some from a backup oﬃce in Rutherford, N.J. Throughout the day, Citi representatives called ﬁnance oﬃcers in state and local government to deliver the bad news: Their short-term borrowing costs were about to spike. And longer deals were on hold. Patrick Brett, head of municipal debt capital markets at the ﬁrm, was making his calls from a rustic house on a forested ridgeline in the Catskills. He booked the Airbnb in March a few days after the head of the Port Authority of New York and New Jersey, a major municipal borrower, conﬁrmed publicly that he had tested positive for coronavirus. That weekend Mr. Brett and his family left Brooklyn in a gray Chevy Suburban so packed with supplies that Mr. Brett’s father-in-law had to balance a 12-pack of paper towels on his lap. From a makeshift oﬃce, he spent Monday in back-to-back phone calls. That night, he would write his ﬁrst of many crisis updates to state and local government ﬁnance oﬃcials around the country. In his mind, this was worse than 2008. “I don’t think anyone alive has experienced anything more violent,” he said in an email to The Wall Street Journal. Citi bankers had reached out to Larry Hammel, ﬁnance chief of the Forsyth County school district, the previous week as the muni market began to dry up. The district had planned to sell nearly $150 million in bonds on March 17 so it could keep construction going on four desperately needed new schools.   When Citi advised putting the deal on hold for a while, Mr. Hammel huddled with his chief facilities oﬃcer and the two men did the math. Without the cash infusion the district had expected from the bonds, construction would halt in July. “It’s one of those days where you just go home and say ‘It’s either going to be beer or wine,’ ” Mr. Hammel said. He began discussions with a local bank about whether the district might be able to secure a bridge loan to keep school construction going. It wasn’t until March 30 that the bonds eventually sold, largely thanks to government programs that brought markets back from the brink. The liquidity panic quickly leaked into the stock market. Thomas Peterﬀy, chairman of Interactive Brokers Group, an electronic brokerage popular with day traders, had trouble sleeping Sunday night. He would wake up, grab his iPhone and get another dire update on where stock futures were trading. They dropped 5%, the most allowed in a single session. By the time Mr. Peterﬀy started work on Monday morning from his home in Palm Beach, Fla., many investors had been forced to sell their positions because they didn’t have enough cash on hand to maintain them. He repeatedly asked his team how big the losses in clients’ accounts were, which bets had soured, and how much money Interactive Brokers could be on the hook for if they didn’t make good. “As the day went on, more and more [positions] were liquidated,” Mr. Peterﬀy said. Adding to the tumult, Mr. Peterﬀy said, bets against volatility. For more than a decade, markets had been generally calm. A wildly popular bet for traders large and small was that  they would remain so. But volatility had been mounting since late February. By March 16, it was at a roar. The Cboe Volatility Index, known as Wall Street’s “fear gauge,” lurched higher during the day and closed at its highest level on record of 82.69. It didn’t help that the virus that morning had closed the trading ﬂoor in Chicago where many options are bought and sold. Old-fashioned trading using shouting and hand signals has dwindled for most markets around the country, but Cboe Global Markets’ open-outcry pits are typically bustling with human traders. Cboe had made the call to close the ﬂoor on Thursday as a precautionary measure, and executives spent Saturday working with brokers to test the all-electronic trading market ahead of its debut on Monday morning. The tests went well, but Sunday’s selloﬀ in stock futures had brought new complications, said Chris Isaacson, Cboe’s chief operating oﬃcer. After S&P 500 futures hit their maximum decline, Cboe opted to delay the premarket trading.  While Cboe had sent most of its employees to work from home that week, Mr. Isaacson went into the ﬁrm’s Kansas City oﬃces that Monday to monitor the market along with the technology and operations staﬀ. The stock futures selloﬀ never let up, so options tied to the same benchmark remained in lockdown all Monday morning, waiting for the 9:30 a.m. opening. By then, Mr. Isaacson and his team knew what was coming next: “The market was going to have a rocky opening,” he said. Some options opened right on time only to be halted one second later, when the selloﬀ on the stock market triggered its circuit-breaker. “It was one of the most intense mornings of my career,” Mr. Isaacson said. Malachite Capital Management, a New York hedge-fund ﬁrm, didn’t make it past Tuesday. On March 17, the ﬁrm said it would shut down, blaming the “extreme adverse market conditions of recent weeks.” The losses were also extreme for others that traded on volatility. At JD Capital Management LLC, a hedge-fund ﬁrm founded by Goldman Sachs veteran J. David Rogers, the ﬁrm’s Tempo Volatility Fund lost 75% or more for the month of March. That same Monday, traders at Allianz Global Investors, a money-management arm of the German insurance giant, were struggling to restructure their own batch of disastrous options trades.  Allianz’s Structured Alpha funds had been a big seller of insurance against a market selloﬀ in the short term and a buyer over the longer term. The strategy produced a steady income, as the fund collected premiums from investors hedging against a downturn. The funds might lose money for a month during a selloﬀ as they restructured those short-term trades, Greg Tournant, the funds’ portfolio manager, said during a May 2016 marketing video, but over time they’d make money. “We are acting like an insurance company, collecting premiums,” Mr. Tournant said. “When there is a catastrophic event, we might have to pay—very much like an insurance company. The positions we buy to protect ourselves from those catastrophic shocks—you could label those as reinsurance.” When the big storm arrived in March, though, the strategy didn’t work. As options contracts swung dramatically, Allianz managers scrambled to restructure their trades. They struggled to keep up; the stock market was spiraling lower at a pace the managers didn’t expect. On March 25, Allianz informed investors that two of its Structured Alpha hedge funds that managed nearly $2.3 billion would be liquidated. Allianz executives told investors that one of the funds was down about 97% since the start of the year, one person familiar with the matter said. Even after a March 25 conference call with Allianz, some investors said, they were still unsure what exactly went wrong. Allianz didn’t tell them how much money they’d get back, or when to expect it.""" 


In [5]:
res = sum_lmodel.fullAnnotate(example_txt)[0]


print ('Summary:', res['summaries'][0].result)

2021-10-19 12:55:02.219523: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 464027648 exceeds 10% of free system memory.
2021-10-19 12:55:02.486446: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 464027648 exceeds 10% of free system memory.
2021-10-19 12:55:02.523090: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 464027648 exceeds 10% of free system memory.
2021-10-19 12:55:02.908023: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 464027648 exceeds 10% of free system memory.
2021-10-19 12:55:03.235775: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 464027648 exceeds 10% of free system memory.


Summary: a swath of interest-rate swap contracts swung in stock markets . the u.s. government has rewritten the rules on money funds . the u.s. government has rewritten the rules on money funds .


In [ ]:
# Q & A 

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

sentence_detector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "en")\
    .setInputCols(["documents"])\
    .setOutputCol("questions")

t5 = T5Transformer()\
    .pretrained("google_t5_small_ssm_nq", 'en')\
    .setInputCols(["questions"])\
    .setOutputCol("answers")\

qa_pp = Pipeline(stages=[
    document_assembler, sentence_detector, t5
])

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[ | ]sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
Download done! Loading the resource.
[OK!]
google_t5_small_ssm_nq download started this may take some time.
Approximate size to download 139 MB
[ | ]google_t5_small_ssm_nq download started this may take some time.
Approximate size to download 139 MB
[ | ]Download done! Loading the resource.
[OK!]


In [7]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = qa_pp.fit(empty_df)
qa_lmodel = LightPipeline(pipeline_model)

questions = ["why did investors sell bonds?",
             "why was the treasury yield below 1%?",
             "what happended with oil stocks?",
             "Who took advantage of the market downturn?",
             "Who is the founder of Microsoft?"
]

res = qa_lmodel.fullAnnotate(questions)


for i, r in enumerate(res):
  print ("Question:", questions[i])
  for sent in r['answers']:
    print ('Answer:\t', sent.result)

Question: why did investors sell bonds?
Answer:	 due to lack of funding
Question: why was the treasury yield below 1%?
Answer:	 due to low ratings
Question: what happended with oil stocks?
Answer:	 declared unconstitutional
Question: Who took advantage of the market downturn?
Answer:	 The Russian SFSR
Question: Who is the founder of Microsoft?
Answer:	 Microsoft founder Bill Gates
